# Signifikanstestning

Innan vi ger en rekommendation så bör vi kontrollera att det rör sig om en statistiskt säkerställd skillnad och inte bara en slumpmässig variation.

![Bounce rate](images/bounce_rate.png)

Vi sätter därför upp en noll-hypotes med antagandet att bounce-raten är samma för besökare som kommer från social media och för besökare som kommer via en annan referral.


## Fishers exakta test

Har man väldigt lite data kan man använda sig av Fishers exakta test: 

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.fisher_exact.html

|       | social | referral | 
|-------|:-------:|:------:|
| bounce | 6 | 6 |
| icke-bounce | 6 | 4 |

Om Fishers test ger ett p-värde < 0.05 så säger vi att skillnaden är signifikant.




In [3]:
!pip install requests --user

import pandas as pd 
import json 
import requests 

requests.get('https://api.github.com')


ImportError: No module named requests

In [0]:
import scipy.stats as stats

oddsratio, pvalue = stats.fisher_exact([[8, 2], [1, 5]])

print (pvalue)